# Exercise - Get the rock sample data into Visual Studio Code

In [53]:
# Needed to enable %%sparksql magic (Enables SQL instructions execution)

%pip install sparksql-magic --quiet
%load_ext sparksql_magic

Note: you may need to restart the kernel to use updated packages.
The sparksql_magic extension is already loaded. To reload it, use:
  %reload_ext sparksql_magic


In [99]:
# Import PySpark
from pyspark.sql import SparkSession

In [100]:
#Create SparkSession
spark = SparkSession.builder.appName("TestSpark") \
    .enableHiveSupport() \
    .getOrCreate()

#    .config("spark.sql.warehouse.dir", "/user/hive/warehouse") \
#    .enableHiveSupport() \
#    .config("spark.sql.warehouse.dir", "/hive/warehouse/dir") \
#    .config("hive.metastore.uris", "thrift://localhost:9083") \



In [101]:
#Get data file

df = spark.read.options(inferSchema='True', header= 'True').csv("file:///C:/Users/manso/LocalDocuments/10-TechProjects/over-the-moon/sample-return/data/rocksamples.csv")

In [102]:
display("Schema:")

df.printSchema

'Schema:'

<bound method DataFrame.printSchema of DataFrame[ID: int, Mission: string, Type: string, Subtype: string, Weight (g): double, Pristine (%): double]>

In [103]:
# Print DataFrame
df.show(10)

+-----+--------+-------+--------+----------+------------+
|   ID| Mission|   Type| Subtype|Weight (g)|Pristine (%)|
+-----+--------+-------+--------+----------+------------+
|10001|Apollo11|   Soil|Unsieved|     125.8|       88.36|
|10002|Apollo11|   Soil|Unsieved|    5629.0|       93.73|
|10003|Apollo11| Basalt|Ilmenite|     213.0|       65.56|
|10004|Apollo11|   Core|Unsieved|      44.8|       71.76|
|10005|Apollo11|   Core|Unsieved|      53.4|       40.31|
|10008|Apollo11|   Soil|Unsieved|      89.0|        5.75|
|10009|Apollo11|Breccia|Regolith|     112.0|       97.27|
|10010|Apollo11|   Soil|Unsieved|     491.0|       91.03|
|10011|Apollo11|   Soil|Unsieved|      82.6|       62.01|
|10014|Apollo11|   Soil|Unsieved|      50.0|         0.0|
+-----+--------+-------+--------+----------+------------+
only showing top 10 rows



# Exercise - Determine the question to ask to inform data cleansing

In [104]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col,lag, lit, round, mean as _mean, sum as _sum


In [105]:
# Convert the sample weight


# Rename Column to remove space
df = df.withColumnRenamed("Weight (g)","Weight(g)")

df = df.withColumn("Weight(kg)", col("Weight(g)") * 0.001)


df.head(10)

[Row(ID=10001, Mission='Apollo11', Type='Soil', Subtype='Unsieved', Weight(g)=125.8, Pristine (%)=88.36, Weight(kg)=0.1258),
 Row(ID=10002, Mission='Apollo11', Type='Soil', Subtype='Unsieved', Weight(g)=5629.0, Pristine (%)=93.73, Weight(kg)=5.6290000000000004),
 Row(ID=10003, Mission='Apollo11', Type='Basalt', Subtype='Ilmenite', Weight(g)=213.0, Pristine (%)=65.56, Weight(kg)=0.213),
 Row(ID=10004, Mission='Apollo11', Type='Core', Subtype='Unsieved', Weight(g)=44.8, Pristine (%)=71.76, Weight(kg)=0.0448),
 Row(ID=10005, Mission='Apollo11', Type='Core', Subtype='Unsieved', Weight(g)=53.4, Pristine (%)=40.31, Weight(kg)=0.0534),
 Row(ID=10008, Mission='Apollo11', Type='Soil', Subtype='Unsieved', Weight(g)=89.0, Pristine (%)=5.75, Weight(kg)=0.089),
 Row(ID=10009, Mission='Apollo11', Type='Breccia', Subtype='Regolith', Weight(g)=112.0, Pristine (%)=97.27, Weight(kg)=0.112),
 Row(ID=10010, Mission='Apollo11', Type='Soil', Subtype='Unsieved', Weight(g)=491.0, Pristine (%)=91.03, Weight(kg

In [106]:
# Create a new DataFrame called missions that will be a summary of data for each of the six Apollo missions that brought samples back. 
# Create a column in this DataFrame called Mission that has one row for each mission.

missions = df.dropDuplicates(["Mission"]).select("Mission")
missions.head(10)


[Row(Mission='Apollo15'),
 Row(Mission='Apollo11'),
 Row(Mission='Apollo14'),
 Row(Mission='Apollo12'),
 Row(Mission='Apollo17'),
 Row(Mission='Apollo16')]

In [107]:
#missions.toPandas().info()

type(missions)

pyspark.sql.dataframe.DataFrame

In [108]:
# Sum total sample weight by mission

sample_total_weight = df.groupby('Mission').sum('Weight(kg)')


# Using Join expression and remove duplicate columns
missions = missions.join(sample_total_weight,missions["Mission"] == sample_total_weight["Mission"]) \
    .select(missions["Mission"], sample_total_weight["sum(Weight(kg))"]) \
    .orderBy(missions["Mission"])

# Rename Column
missions = missions.withColumnRenamed("sum(Weight(kg))","Sample_weight(kg)")
missions.show()


+--------+------------------+
| Mission| Sample_weight(kg)|
+--------+------------------+
|Apollo11|          21.55424|
|Apollo12|          34.34238|
|Apollo14|          41.83363|
|Apollo15| 75.39910000000005|
|Apollo16| 92.46262000000006|
|Apollo17|109.44402000000001|
+--------+------------------+



In [109]:
# Get the difference in weights across missions

# Create window
windowSpec  = Window.orderBy("Mission")

#Simulate Pandas diff() API on PySpark usinl lag function (with above windowSpec)
missions = missions.withColumn("lag",lag("Sample_weight(kg)",1).over(windowSpec)) \
      .withColumn("Weight_diff", col("Sample_weight(kg)") - col("lag")) \
      .select("Mission","Sample_weight(kg)","Weight_diff")


In [110]:
# Replace Null values

missions = missions.na.fill(value=0,subset=["Weight_diff"])
missions.show()


+--------+------------------+------------------+
| Mission| Sample_weight(kg)|       Weight_diff|
+--------+------------------+------------------+
|Apollo11|          21.55424|               0.0|
|Apollo12|          34.34238|12.788139999999999|
|Apollo14|          41.83363| 7.491250000000001|
|Apollo15| 75.39910000000005| 33.56547000000005|
|Apollo16| 92.46262000000006| 17.06352000000001|
|Apollo17|109.44402000000001| 16.98139999999995|
+--------+------------------+------------------+



In [111]:
%%sparksql

CREATE DATABASE IF NOT EXISTS train

In [112]:
# Create train database 
spark.sql("CREATE DATABASE IF NOT EXISTS train")

# Write to (managed) tables on train database
df.write.mode("Overwrite").saveAsTable("train.rockSamples")
missions.write.mode("Overwrite") \
    .option("path", "/user/hive/warehouse/train.db/missions") \
    .saveAsTable("train.Missions")

In [113]:
# List existing databases
spark.sql('show databases').show()

# List train database tables
spark.catalog.listTables('train')

+---------+
|namespace|
+---------+
|  default|
|    train|
+---------+



[Table(name='missions', catalog='spark_catalog', namespace=['train'], description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='rocksamples', catalog='spark_catalog', namespace=['train'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='AuxCM', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='AuxLM', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='AuxMissions', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='result', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [114]:
%%sparksql --cache --view result dfRes
SELECT * FROM train.Missions WHERE Mission='Apollo14'



cache dataframe with lazy load
create temporary view `result`
capture dataframe to local variable `dfRes`


Mission,Sample_weight(kg),Weight_diff
Apollo14,41.83363,7.491250000000001


In [115]:
dfRes.show()

+--------+-----------------+-----------------+
| Mission|Sample_weight(kg)|      Weight_diff|
+--------+-----------------+-----------------+
|Apollo14|         41.83363|7.491250000000001|
+--------+-----------------+-----------------+



In [116]:
# Test previous saved tables
spark.sql("SELECT * FROM train.Missions WHERE Mission='Apollo14'").show()

+--------+-----------------+-----------------+
| Mission|Sample_weight(kg)|      Weight_diff|
+--------+-----------------+-----------------+
|Apollo14|         41.83363|7.491250000000001|
+--------+-----------------+-----------------+



# Exercise - Add rocket weight data to the mission analysis

In [117]:
# Add in command and lunar module data

lunarModuleData = [("Eagle (LM-5)",15103), 
        ("Intrepid (LM-6)",15235), 
        ("Antares (LM-8)",15264), 
        ("Falcon (LM-10)",16430),
        ("Orion (LM-11)",16445),
        ("Challenger (LM-12)",16456)
      ]

commandModuleData = [("Columbia (CSM-107)",5560), 
        ("Yankee Clipper (CM-108)",5609), 
        ("Kitty Hawk (CM-110)",5758), 
        ("Endeavor (CM-112)",5875),
        ("Casper (CM-113)",5840),
        ("America (CM-114)",5960)
      ]


# Generate ID Column on Dataframes to be able to join them after
dfLM = spark.createDataFrame(lunarModuleData,["Lunar_module(LM)", "LM_mass(kg)"])
dfLM.createOrReplaceTempView('AuxLM')
dfLM = spark.sql('select ROW_NUMBER() OVER(ORDER BY (Select 0)) AS Row_Num, * from AuxLM')

#dfLM.show()

dfCM = spark.createDataFrame(commandModuleData, ["Command_module(CM)", "CM_mass(kg)"])
dfCM.createOrReplaceTempView('AuxCM')
dfCM = spark.sql('select ROW_NUMBER() OVER(ORDER BY (Select 0)) AS Row_Num, * from AuxCM')

#dfCM.show()

dfMissions = missions.select("Mission")
dfMissions.createOrReplaceTempView('AuxMissions')
dfMissions = spark.sql('select ROW_NUMBER() OVER(ORDER BY (Select 0)) AS Row_Num, * from AuxMissions')

#dfMissions.show()

# Join Dataframes using ID column
dfAll = dfMissions.join(dfLM,dfMissions["Row_Num"] == dfLM["Row_Num"]) \
        .drop(dfMissions["Row_Num"])
      
dfAll = dfAll.join(dfCM,dfAll["Row_Num"] == dfCM["Row_Num"]) \
        .drop(dfAll["Row_Num"], dfCM["Row_Num"])


#dfAll.printSchema()
#dfAll.show()

# Update missions Dataframe with the new fields
missions = missions.join(dfAll,missions["Mission"] == dfAll["Mission"]) \
        .drop(dfAll["Mission"])


missions.printSchema()
missions.show()

root
 |-- Sample_weight(kg): double (nullable = true)
 |-- Weight_diff: double (nullable = false)
 |-- Mission: string (nullable = true)
 |-- Lunar_module(LM): string (nullable = true)
 |-- LM_mass(kg): long (nullable = true)
 |-- Command_module(CM): string (nullable = true)
 |-- CM_mass(kg): long (nullable = true)

+------------------+------------------+--------+------------------+-----------+--------------------+-----------+
| Sample_weight(kg)|       Weight_diff| Mission|  Lunar_module(LM)|LM_mass(kg)|  Command_module(CM)|CM_mass(kg)|
+------------------+------------------+--------+------------------+-----------+--------------------+-----------+
|          21.55424|               0.0|Apollo11|      Eagle (LM-5)|      15103|  Columbia (CSM-107)|       5560|
|109.44402000000001| 16.98139999999995|Apollo17|Challenger (LM-12)|      16456|    America (CM-114)|       5960|
|          41.83363| 7.491250000000001|Apollo14|    Antares (LM-8)|      15264| Kitty Hawk (CM-110)|       5758|
| 92

In [118]:
# Get the difference in weights across missions

# Create window
windowSpec  = Window.orderBy("Mission")

#Simulate Pandas diff() API on PySpark usinl lag function (with above windowSpec)
missions = missions.withColumn("lag",lag("LM_mass(kg)",1).over(windowSpec)) \
      .withColumn("LM_mass_diff", col("LM_mass(kg)") - col("lag")) \
      .drop("lag")


missions = missions.withColumn("lag",lag("CM_mass(kg)",1).over(windowSpec)) \
      .withColumn("CM_mass_diff", col("CM_mass(kg)") - col("lag")) \
      .drop("lag")

# Replace Null values
missions = missions.na.fill(value=0,subset=["LM_mass_diff"])
missions = missions.na.fill(value=0,subset=["CM_mass_diff"])


missions.printSchema()
missions.show()

root
 |-- Sample_weight(kg): double (nullable = true)
 |-- Weight_diff: double (nullable = false)
 |-- Mission: string (nullable = true)
 |-- Lunar_module(LM): string (nullable = true)
 |-- LM_mass(kg): long (nullable = true)
 |-- Command_module(CM): string (nullable = true)
 |-- CM_mass(kg): long (nullable = true)
 |-- LM_mass_diff: long (nullable = true)
 |-- CM_mass_diff: long (nullable = true)

+------------------+------------------+--------+------------------+-----------+--------------------+-----------+------------+------------+
| Sample_weight(kg)|       Weight_diff| Mission|  Lunar_module(LM)|LM_mass(kg)|  Command_module(CM)|CM_mass(kg)|LM_mass_diff|CM_mass_diff|
+------------------+------------------+--------+------------------+-----------+--------------------+-----------+------------+------------+
|          21.55424|               0.0|Apollo11|      Eagle (LM-5)|      15103|  Columbia (CSM-107)|       5560|           0|           0|
|          34.34238|12.788139999999999|Apo

In [119]:
# Add some totals for each mission across both the lunar and command modules

missions = missions.withColumn("Total_weight(kg)", col("LM_mass(kg)") + col("CM_mass(kg)")) \
      .withColumn("Total_weight_diff", col("LM_mass_diff") + col("CM_mass_diff")) \


missions.printSchema()
missions.show()


root
 |-- Sample_weight(kg): double (nullable = true)
 |-- Weight_diff: double (nullable = false)
 |-- Mission: string (nullable = true)
 |-- Lunar_module(LM): string (nullable = true)
 |-- LM_mass(kg): long (nullable = true)
 |-- Command_module(CM): string (nullable = true)
 |-- CM_mass(kg): long (nullable = true)
 |-- LM_mass_diff: long (nullable = true)
 |-- CM_mass_diff: long (nullable = true)
 |-- Total_weight(kg): long (nullable = true)
 |-- Total_weight_diff: long (nullable = true)

+------------------+------------------+--------+------------------+-----------+--------------------+-----------+------------+------------+----------------+-----------------+
| Sample_weight(kg)|       Weight_diff| Mission|  Lunar_module(LM)|LM_mass(kg)|  Command_module(CM)|CM_mass(kg)|LM_mass_diff|CM_mass_diff|Total_weight(kg)|Total_weight_diff|
+------------------+------------------+--------+------------------+-----------+--------------------+-----------+------------+------------+----------------+--

# Exercise - Understand the data in the missions DataFrame

In [120]:
# Sample-to-weight ratio

saturnVPayload = 43500

missions = missions.withColumn("Crewed_area_Payload", col("Total_weight(kg)") / saturnVPayload) \
    .withColumn("Sample_Crewed_area", col("Sample_weight(kg)") / col("Total_weight(kg)")) \
    .withColumn("Sample_Payload", round(col("Sample_weight(kg)") / saturnVPayload,6))

# Round all numeric columns to 6 decimal places
for c_name, c_type in missions.dtypes:
    if c_type in ('long','double', 'float'):
        missions = missions.withColumn(c_name, round(c_name, 6))

missions.printSchema()
missions.show()


root
 |-- Sample_weight(kg): double (nullable = true)
 |-- Weight_diff: double (nullable = true)
 |-- Mission: string (nullable = true)
 |-- Lunar_module(LM): string (nullable = true)
 |-- LM_mass(kg): long (nullable = true)
 |-- Command_module(CM): string (nullable = true)
 |-- CM_mass(kg): long (nullable = true)
 |-- LM_mass_diff: long (nullable = true)
 |-- CM_mass_diff: long (nullable = true)
 |-- Total_weight(kg): long (nullable = true)
 |-- Total_weight_diff: long (nullable = true)
 |-- Crewed_area_Payload: double (nullable = true)
 |-- Sample_Crewed_area: double (nullable = true)
 |-- Sample_Payload: double (nullable = true)

+-----------------+-----------+--------+------------------+-----------+--------------------+-----------+------------+------------+----------------+-----------------+-------------------+------------------+--------------+
|Sample_weight(kg)|Weight_diff| Mission|  Lunar_module(LM)|LM_mass(kg)|  Command_module(CM)|CM_mass(kg)|LM_mass_diff|CM_mass_diff|Total_wei

In [121]:
# Update (managed) Missions table on train database

missions.write.mode("Overwrite") \
    .option("path", "/user/hive/warehouse/train.db/missions") \
    .saveAsTable("train.Missions")

spark.sql("Select * From train.Missions;").show()
spark.sql("DESCRIBE TABLE train.Missions;").show()

+-----------------+-----------+--------+------------------+-----------+--------------------+-----------+------------+------------+----------------+-----------------+-------------------+------------------+--------------+
|Sample_weight(kg)|Weight_diff| Mission|  Lunar_module(LM)|LM_mass(kg)|  Command_module(CM)|CM_mass(kg)|LM_mass_diff|CM_mass_diff|Total_weight(kg)|Total_weight_diff|Crewed_area_Payload|Sample_Crewed_area|Sample_Payload|
+-----------------+-----------+--------+------------------+-----------+--------------------+-----------+------------+------------+----------------+-----------------+-------------------+------------------+--------------+
|         21.55424|        0.0|Apollo11|      Eagle (LM-5)|      15103|  Columbia (CSM-107)|       5560|           0|           0|           20663|                0|           0.475011|          0.001043|       4.95E-4|
|         34.34238|   12.78814|Apollo12|   Intrepid (LM-6)|      15235|Yankee Clipper (C...|       5609|         132|   

In [122]:
# Take the average of all those ratios across all the missions

# Note: (...).collect()[0][0] was used to allow a single value in variable instead of dataframe
crewedArea_payload_ratio = missions.select(round(_mean("Crewed_area_Payload"),6)).collect()[0][0]
sample_crewedArea_ratio = missions.select(round(_mean("Sample_Crewed_area"),6)).collect()[0][0]
sample_payload_ratio = missions.select(round(_mean(col("Sample_Payload")),6)).collect()[0][0]

# Exercise - Predict Artemis sample capacity

In [123]:
# Create an Artemis mission DataFrame
artemis_crewedArea = 26520

missionData = ["artemis1","artemis1b","artemis2"]
weightData = [artemis_crewedArea,artemis_crewedArea,artemis_crewedArea]
payloadData = [26988, 37965, 42955]

# joins / Zip the 3 configuration lists
artemisAllData = zip(missionData,weightData,payloadData)

# Convert zip object to a list
artemisAllData = list(artemisAllData)

#print(artemisAllData)
print(*artemisAllData)

# Schema configuration
schemaArtemis = ["Mission", "Total_weight(kg)", "Payload(kg)"]

artemis_mission = spark.createDataFrame(data = artemisAllData, schema = schemaArtemis)

artemis_mission.printSchema()
artemis_mission.show()


('artemis1', 26520, 26988) ('artemis1b', 26520, 37965) ('artemis2', 26520, 42955)
root
 |-- Mission: string (nullable = true)
 |-- Total_weight(kg): long (nullable = true)
 |-- Payload(kg): long (nullable = true)

+---------+----------------+-----------+
|  Mission|Total_weight(kg)|Payload(kg)|
+---------+----------------+-----------+
| artemis1|           26520|      26988|
|artemis1b|           26520|      37965|
| artemis2|           26520|      42955|
+---------+----------------+-----------+



In [124]:
# Estimate the weight of samples based on the ratios we determined from the Apollo missions

artemis_mission = artemis_mission.withColumn("Sample_weight_from_total(kg)",round(col("Total_weight(kg)") * sample_crewedArea_ratio,6))
artemis_mission = artemis_mission.withColumn("Sample_weight_from_payload(kg)",round(col("Payload(kg)") * sample_payload_ratio,6))


artemis_mission.printSchema()
artemis_mission.show()

root
 |-- Mission: string (nullable = true)
 |-- Total_weight(kg): long (nullable = true)
 |-- Payload(kg): long (nullable = true)
 |-- Sample_weight_from_total(kg): double (nullable = true)
 |-- Sample_weight_from_payload(kg): double (nullable = true)

+---------+----------------+-----------+----------------------------+------------------------------+
|  Mission|Total_weight(kg)|Payload(kg)|Sample_weight_from_total(kg)|Sample_weight_from_payload(kg)|
+---------+----------------+-----------+----------------------------+------------------------------+
| artemis1|           26520|      26988|                    75.55548|                     38.781756|
|artemis1b|           26520|      37965|                    75.55548|                     54.555705|
| artemis2|           26520|      42955|                    75.55548|                     61.726335|
+---------+----------------+-----------+----------------------------+------------------------------+



In [125]:
# Get the average of the two predictions:
artemis_mission = artemis_mission.withColumn("Estimated_sample_weight(kg)", round((col("Sample_weight_from_payload(kg)") + col("Sample_weight_from_total(kg)"))/2,6))

artemis_mission.describe().show()
artemis_mission.printSchema()
artemis_mission.show()


+-------+--------+----------------+------------------+----------------------------+------------------------------+---------------------------+
|summary| Mission|Total_weight(kg)|       Payload(kg)|Sample_weight_from_total(kg)|Sample_weight_from_payload(kg)|Estimated_sample_weight(kg)|
+-------+--------+----------------+------------------+----------------------------+------------------------------+---------------------------+
|  count|       3|               3|                 3|                           3|                             3|                          3|
|   mean|    NULL|         26520.0|35969.333333333336|                    75.55548|            51.687931999999996|          63.62170633333333|
| stddev|    NULL|             0.0| 8168.432305732437|                         0.0|             11.73803722333751|          5.869018886548102|
|    min|artemis1|           26520|             26988|                    75.55548|                     38.781756|                  57.168618|

# Exercise - Prioritize Moon rock sample gathering based on data

In [126]:
# Determine how much remains of each sample that was returned from the Apollo missions by multiplying the weight of the sample that was originally collected 
# by the percentage of remaining pristine sample.

rock_samples = df

rock_samples = rock_samples.withColumn("Remaining(kg)", round(col("Weight(kg)") * (col("Pristine (%)") * .01), 6))

rock_samples.printSchema()
rock_samples.show(5)


root
 |-- ID: integer (nullable = true)
 |-- Mission: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Subtype: string (nullable = true)
 |-- Weight(g): double (nullable = true)
 |-- Pristine (%): double (nullable = true)
 |-- Weight(kg): double (nullable = true)
 |-- Remaining(kg): double (nullable = true)

+-----+--------+------+--------+---------+------------+------------------+-------------+
|   ID| Mission|  Type| Subtype|Weight(g)|Pristine (%)|        Weight(kg)|Remaining(kg)|
+-----+--------+------+--------+---------+------------+------------------+-------------+
|10001|Apollo11|  Soil|Unsieved|    125.8|       88.36|            0.1258|     0.111157|
|10002|Apollo11|  Soil|Unsieved|   5629.0|       93.73|5.6290000000000004|     5.276062|
|10003|Apollo11|Basalt|Ilmenite|    213.0|       65.56|             0.213|     0.139643|
|10004|Apollo11|  Core|Unsieved|     44.8|       71.76|            0.0448|     0.032148|
|10005|Apollo11|  Core|Unsieved|     53.4|       4

In [127]:
# DataFrame isn't useful for examining more than 2,000 samples. To get a better understanding of what the dataset contains, you can use the describe() function:

rock_samples.describe().show()

+-------+-----------------+--------+-------+---------+------------------+------------------+-------------------+-------------------+
|summary|               ID| Mission|   Type|  Subtype|         Weight(g)|      Pristine (%)|         Weight(kg)|      Remaining(kg)|
+-------+-----------------+--------+-------+---------+------------------+------------------+-------------------+-------------------+
|  count|             2229|    2229|   2229|     2226|              2229|              2229|               2229|               2229|
|   mean|52058.43203230148|    NULL|   NULL|     NULL|168.25302377747855|  84.5127635711082|0.16825302377747908|0.13810338941229278|
| stddev|26207.65147099608|    NULL|   NULL|     NULL| 637.2864579561242|22.057298716073344| 0.6372864579561255| 0.5259540237327816|
|    min|            10001|Apollo11| Basalt|    1-2mm|               0.0|               0.0|                0.0|                0.0|
|    max|            79537|Apollo17|Special|unstudied|           1172

In [128]:
# On average, each sample weighs about .16 kg and has about 84% of the original amount remaining. 
# We can use this knowledge to determine which samples are likely running low

low_samples = rock_samples.where((col("Weight(kg)") >= .16) & (col("Pristine (%)") <= 50))

low_samples.printSchema()
low_samples.show(5)


root
 |-- ID: integer (nullable = true)
 |-- Mission: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Subtype: string (nullable = true)
 |-- Weight(g): double (nullable = true)
 |-- Pristine (%): double (nullable = true)
 |-- Weight(kg): double (nullable = true)
 |-- Remaining(kg): double (nullable = true)

+-----+--------+-------+--------+---------+------------+----------+-------------+
|   ID| Mission|   Type| Subtype|Weight(g)|Pristine (%)|Weight(kg)|Remaining(kg)|
+-----+--------+-------+--------+---------+------------+----------+-------------+
|10017|Apollo11| Basalt|Ilmenite|    973.0|       43.71|     0.973|     0.425298|
|10020|Apollo11| Basalt|Ilmenite|    425.0|       27.88|     0.425|      0.11849|
|10021|Apollo11|Breccia|Regolith|    250.0|       30.21|      0.25|     0.075525|
|10045|Apollo11| Basalt| Olivine|    185.0|       12.13|     0.185|     0.022441|
|10057|Apollo11| Basalt|Ilmenite|    919.0|       35.15|     0.919|     0.323029|
+-----+--------+-

In [129]:
low_samples.toPandas().info()

type(low_samples)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             27 non-null     int32  
 1   Mission        27 non-null     object 
 2   Type           27 non-null     object 
 3   Subtype        27 non-null     object 
 4   Weight(g)      27 non-null     float64
 5   Pristine (%)   27 non-null     float64
 6   Weight(kg)     27 non-null     float64
 7   Remaining(kg)  27 non-null     float64
dtypes: float64(4), int32(1), object(3)
memory usage: 1.7+ KB


pyspark.sql.dataframe.DataFrame

In [130]:
# See how many unique types we have across the low_samples and rock_samples DataFrames.

dfUniqueLow = low_samples.select("Type").distinct()
dfUniqueRockS = rock_samples.select("Type").distinct()

dfUniqueLowTypes = dfUniqueLow.union(dfUniqueRockS) \
            .distinct()

dfUniqueLowTypes.printSchema()
dfUniqueLowTypes.show()


root
 |-- Type: string (nullable = true)

+-------+
|   Type|
+-------+
|   Soil|
| Basalt|
|Breccia|
|   Core|
|Special|
|Crustal|
+-------+



In [131]:
# In our low_samples DataFrame, how many of each type are considered low?

low_samples.groupby("Type") \
    .count() \
    .orderBy("Type") \
    .show()



+-------+-----+
|   Type|count|
+-------+-----+
| Basalt|   14|
|Breccia|    8|
|   Core|    1|
|   Soil|    4|
+-------+-----+



In [132]:
# Focus on the Basalt and Breccia rock types for the samples that we need to have collected:

# Test - In case all Types were to be considered
# auxFilterTypes = list(dfUniqueLowTypes.select("Type"))

# Filter fot specific Types (list)
auxFilterTypes = ["Basalt","Breccia"]

needed_samples = low_samples.where(col("Type").isin(auxFilterTypes))

needed_samples.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             22 non-null     int32  
 1   Mission        22 non-null     object 
 2   Type           22 non-null     object 
 3   Subtype        22 non-null     object 
 4   Weight(g)      22 non-null     float64
 5   Pristine (%)   22 non-null     float64
 6   Weight(kg)     22 non-null     float64
 7   Remaining(kg)  22 non-null     float64
dtypes: float64(4), int32(1), object(3)
memory usage: 1.4+ KB


# Exercise - Develop a recommendation of Moon rock samples to be collected

In [133]:
# Compare the total weight from the needed_samples DataFrame to the rock_samples DataFrame

needed_samples.groupby("Type") \
    .sum("Weight(kg)") \
    .orderBy("Type") \
    .show()


rock_samples.groupby("Type") \
    .sum("Weight(kg)") \
    .orderBy("Type") \
    .show()



+-------+---------------+
|   Type|sum(Weight(kg))|
+-------+---------------+
| Basalt|        17.4234|
|Breccia|        10.1185|
+-------+---------------+

+-------+------------------+
|   Type|   sum(Weight(kg))|
+-------+------------------+
| Basalt| 93.14076999999995|
|Breccia|168.88074999999992|
|   Core|19.935870000000005|
|Crustal|           4.74469|
|   Soil| 87.58980999999994|
|Special|            0.7441|
+-------+------------------+



In [134]:
# Add Crustal rocks to the set of needed samples

crustalSample = rock_samples.where(col("Type") == 'Crustal')

crustalSample.show(10)

needed_samples = needed_samples.union(crustalSample)

needed_samples.printSchema()
needed_samples.toPandas().info()


+-----+--------+-------+-----------+---------+------------+--------------------+-------------+
|   ID| Mission|   Type|    Subtype|Weight(g)|Pristine (%)|          Weight(kg)|Remaining(kg)|
+-----+--------+-------+-----------+---------+------------+--------------------+-------------+
|15361|Apollo15|Crustal|Cataclastic|      0.9|       66.56|9.000000000000001E-4|      5.99E-4|
|15362|Apollo15|Crustal|Cataclastic|      4.2|       56.88|0.004200000000000001|     0.002389|
|15363|Apollo15|Crustal|Cataclastic|      0.5|        71.0|              5.0E-4|      3.55E-4|
|15415|Apollo15|Crustal|Anorthosite|    269.4|       67.07|              0.2694|     0.180687|
|15437|Apollo15|Crustal|Anorthosite|      1.0|        80.0|               0.001|       8.0E-4|
|60057|Apollo16|Crustal|Anorthosite|      3.1|       100.0|0.003100000000000...|       0.0031|
|60059|Apollo16|Crustal|Anorthosite|     1.05|       100.0|0.001050000000000...|      0.00105|
|60619|Apollo16|Crustal|Anorthosite|     28.0|    

In [135]:
# We need a column for each type of rock that we want more samples of:

needed_samples_overview = needed_samples.select("Type").distinct()

needed_samples_overview.show()

+-------+
|   Type|
+-------+
| Basalt|
|Breccia|
|Crustal|
+-------+



In [136]:
# Next, we want the total weight of each type of rock that was originally collected:

needed_sample_weights = needed_samples.groupby("Type") \
    .sum("Weight(kg)") \
    .orderBy("Type") 


needed_samples_overview = needed_samples_overview.join(needed_sample_weights,needed_samples_overview["Type"] == needed_sample_weights["Type"]) \
        .withColumnRenamed("sum(Weight(kg))", "Total_weight(kg)") \
        .drop(needed_samples_overview["Type"]) 
        
        
needed_samples_overview.printSchema()
needed_samples_overview.show()



root
 |-- Type: string (nullable = true)
 |-- Total_weight(kg): double (nullable = true)

+-------+----------------+
|   Type|Total_weight(kg)|
+-------+----------------+
| Basalt|         17.4234|
|Breccia|         10.1185|
|Crustal|         4.74469|
+-------+----------------+



In [137]:
# When astronauts are up on the Moon, one way they can identify rocks is by their size. 
# If we can give them an estimated size of each type of rock that might make their collection process easier.

needed_sample_ave_weights = needed_samples.groupby("Type") \
    .mean("Weight(kg)") \
    .withColumnRenamed("avg(Weight(kg))", "Average_weight(kg)") \
    .orderBy("Type") 

needed_samples_overview = needed_samples_overview.join(needed_sample_ave_weights,needed_samples_overview["Type"] == needed_sample_ave_weights["Type"]) \
        .withColumnRenamed("avg(Weight(kg))", "Average_weight(kg)") \
        .drop(needed_samples_overview["Type"])

needed_samples_overview.printSchema()
needed_samples_overview.show()


root
 |-- Total_weight(kg): double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Average_weight(kg): double (nullable = true)

+----------------+-------+-------------------+
|Total_weight(kg)|   Type| Average_weight(kg)|
+----------------+-------+-------------------+
|         17.4234| Basalt| 1.2445285714285714|
|         10.1185|Breccia|          1.2648125|
|         4.74469|Crustal|0.10314543478260871|
+----------------+-------+-------------------+



In [138]:
# For the three types we're looking for, we should grab the total number we have of each type and get the remaining percentage of each type of rock.

total_rock_count = rock_samples.groupby("Type") \
    .count() \
    .orderBy("Type") \
    .withColumnRenamed("count", "Number_of_samples")

needed_samples_overview = needed_samples_overview.join(total_rock_count,needed_samples_overview["Type"] == total_rock_count["Type"]) \
        .drop(needed_samples_overview["Type"]) 

total_rocks = needed_samples_overview.select(_sum("Number_of_samples")).collect()[0][0]

needed_samples_overview = needed_samples_overview.withColumn("Percentage_of_rocks",round(col("Number_of_samples") / total_rocks,6))

needed_samples_overview.printSchema()
needed_samples_overview.show()


root
 |-- Total_weight(kg): double (nullable = true)
 |-- Average_weight(kg): double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Number_of_samples: long (nullable = false)
 |-- Percentage_of_rocks: double (nullable = true)

+----------------+-------------------+-------+-----------------+-------------------+
|Total_weight(kg)| Average_weight(kg)|   Type|Number_of_samples|Percentage_of_rocks|
+----------------+-------------------+-------+-----------------+-------------------+
|         17.4234| 1.2445285714285714| Basalt|              351|            0.25885|
|         10.1185|          1.2648125|Breccia|              959|           0.707227|
|         4.74469|0.10314543478260871|Crustal|               46|           0.033923|
+----------------+-------------------+-------+-----------------+-------------------+



In [139]:
# Determine the average weight of samples we estimated in the preceding unit.

artemis_ave_weight = artemis_mission.select(_mean("Estimated_sample_weight(kg)")).collect()[0][0]

display(artemis_ave_weight)


63.62170633333333

In [140]:
# We can use this number to determine how many of each rock we want the astronauts to aim to collect:

needed_samples_overview.toPandas().info()

needed_samples_overview = needed_samples_overview.withColumn("Weight_to_collect",round(col("Percentage_of_rocks") * artemis_ave_weight,6))
needed_samples_overview = needed_samples_overview.withColumn("Rocks_to_collect",round(col("Weight_to_collect") / col("Average_weight(kg)"),6))

#needed_samples_overview.printSchema()
needed_samples_overview.show()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Total_weight(kg)     3 non-null      float64
 1   Average_weight(kg)   3 non-null      float64
 2   Type                 3 non-null      object 
 3   Number_of_samples    3 non-null      int64  
 4   Percentage_of_rocks  3 non-null      float64
dtypes: float64(3), int64(1), object(1)
memory usage: 252.0+ bytes
+----------------+-------------------+-------+-----------------+-------------------+-----------------+----------------+
|Total_weight(kg)| Average_weight(kg)|   Type|Number_of_samples|Percentage_of_rocks|Weight_to_collect|Rocks_to_collect|
+----------------+-------------------+-------+-----------------+-------------------+-----------------+----------------+
|         17.4234| 1.2445285714285714| Basalt|              351|            0.25885|        16.468479|       13.232705|
|  

## %%sparksql magic SQL tests

In [175]:
%%sparksql --view vResults dfResult

-- select * from train.Missions;

select 
    NTILE(2) OVER (ORDER BY "m.LM_mass(kg)") rk,
    m.*
from train.Missions m;


create temporary view `vResults`
capture dataframe to local variable `dfResult`


rk,Sample_weight(kg),Weight_diff,Mission,Lunar_module(LM),LM_mass(kg),Command_module(CM),CM_mass(kg),LM_mass_diff,CM_mass_diff,Total_weight(kg),Total_weight_diff,Crewed_area_Payload,Sample_Crewed_area,Sample_Payload
1,21.55424,0.0,Apollo11,Eagle (LM-5),15103,Columbia (CSM-107),5560,0,0,20663,0,0.475011,0.001043,0.000495
1,34.34238,12.78814,Apollo12,Intrepid (LM-6),15235,Yankee Clipper (CM-108),5609,132,49,20844,181,0.479172,0.001648,0.000789
1,41.83363,7.49125,Apollo14,Antares (LM-8),15264,Kitty Hawk (CM-110),5758,29,149,21022,178,0.483264,0.00199,0.000962
2,75.3991,33.56547,Apollo15,Falcon (LM-10),16430,Endeavor (CM-112),5875,1166,117,22305,1283,0.512759,0.00338,0.001733
2,92.46262,17.06352,Apollo16,Orion (LM-11),16445,Casper (CM-113),5840,15,-35,22285,-20,0.512299,0.004149,0.002126
2,109.44402,16.9814,Apollo17,Challenger (LM-12),16456,America (CM-114),5960,11,120,22416,131,0.51531,0.004882,0.002516


In [176]:
%%sparksql

select *
from vResults

rk,Sample_weight(kg),Weight_diff,Mission,Lunar_module(LM),LM_mass(kg),Command_module(CM),CM_mass(kg),LM_mass_diff,CM_mass_diff,Total_weight(kg),Total_weight_diff,Crewed_area_Payload,Sample_Crewed_area,Sample_Payload
1,21.55424,0.0,Apollo11,Eagle (LM-5),15103,Columbia (CSM-107),5560,0,0,20663,0,0.475011,0.001043,0.000495
1,34.34238,12.78814,Apollo12,Intrepid (LM-6),15235,Yankee Clipper (CM-108),5609,132,49,20844,181,0.479172,0.001648,0.000789
1,41.83363,7.49125,Apollo14,Antares (LM-8),15264,Kitty Hawk (CM-110),5758,29,149,21022,178,0.483264,0.00199,0.000962
2,75.3991,33.56547,Apollo15,Falcon (LM-10),16430,Endeavor (CM-112),5875,1166,117,22305,1283,0.512759,0.00338,0.001733
2,92.46262,17.06352,Apollo16,Orion (LM-11),16445,Casper (CM-113),5840,15,-35,22285,-20,0.512299,0.004149,0.002126
2,109.44402,16.9814,Apollo17,Challenger (LM-12),16456,America (CM-114),5960,11,120,22416,131,0.51531,0.004882,0.002516
